In [2]:
import pandas as pd
import numpy as np

In [3]:
# loading imdb actor information
actors_raw = pd.read_csv('data/name.basics.tsv', sep='\t')

# filter out rows with missing data cruical for analysis
actors_raw = actors_raw[actors_raw["birthYear"] != r"\N"]

# loading imdb's principals
principals_raw = pd.read_csv('data/title.principals.tsv', sep='\t')

In [4]:
print("Number of actors:", actors_raw.size)
actors_raw

Number of actors: 3411282


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0045537"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0056404,tt0057345,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0077975,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"
...,...,...,...,...,...,...
12148488,nm9993436,Frank J. Gaily,1915,2008,sound_department,tt0189339
12148570,nm9993526,Ben Ray Lujan,1972,\N,NaN,"tt0476038,tt7516996,tt15385660,tt4209386"
12148578,nm9993535,Henry Lawfull,2006,\N,actor,"tt10187208,tt5900600"
12148699,nm9993675,Ebrahim Alkazi,1925,2020,NaN,\N


In [5]:
#print("Number of movies considered in 'principals' data set", len(list(dict.fromkeys(principals["tconst"]))))
principals_raw

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
53573728,tt9916880,4,nm10535738,actress,\N,"[""Horrid Henry""]"
53573729,tt9916880,5,nm0996406,director,principal director,\N
53573730,tt9916880,6,nm1482639,writer,\N,\N
53573731,tt9916880,7,nm2586970,writer,books,\N


## Filter for leading actors and add column for gender
Filter for leading actors based on imdb's principals data which contains for each movie the most important people (i. e. actors, actress, directors etc.). The importance is given by imdb and stored in the 'ordering' column

In [6]:
# filter for actors and actress in principals
principals = principals_raw[(principals_raw["category"] == "actress") | (principals_raw["category"] == "actor")] 

# add gender column (0=male, 1=female)
principals["gender"] = 0
principals.loc[principals["category"] == "actress", "gender"] = 1

# select ony important columns of principal and actors data set and join them left
principal_actors = principals[["tconst", "ordering", "nconst", "gender"]].merge(actors_raw[["nconst",	"primaryName",	"birthYear", "deathYear"]], how="left", on="nconst")

principal_actors

/var/folders/dk/kt0tgs8j1cb1k_hn1_h2bch80000gn/T/ipykernel_4395/82935873.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principals["gender"] = 0


,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear
0,tt0000005,1,nm0443482,0,Charles Kayser,1878,1966
1,tt0000005,2,nm0653042,0,John Ott,1850,1931
2,tt0000007,1,nm0179163,0,James J. Corbett,1866,1933
3,tt0000007,2,nm0183947,0,Peter Courtney,1867,1896
4,tt0000008,1,nm0653028,0,Fred Ott,1860,1936
...,...,...,...,...,...,...,...
20986815,tt9916856,4,nm10538647,1,NaN,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN
20986817,tt9916880,2,nm0254176,1,NaN,NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN


In [7]:
# Find total number of movies played by every leading actor
unique_actors = principal_actors["nconst"].unique()

male_actors = principal_actors[principal_actors["gender"] == 0]["nconst"].unique()
female_actors = principal_actors[principal_actors["gender"] == 1]["nconst"].unique()
print("There's a total of", len(unique_actors), "unique actors.")

print("There's a total of", len(male_actors), "male actors.")
print("There's a total of", len(female_actors), "female actors.")

There's a total of 2307789 unique actors.
There's a total of 1435557 male actors.
There's a total of 872781 female actors.


In [8]:
total_actor_experience = principal_actors.groupby(["nconst"]).count()

total_actor_experience

,tconst,ordering,gender,primaryName,birthYear,deathYear
nconst,,,,,,
nm0000001,56,56,56,56,56,56
nm0000002,64,64,64,64,64,64
nm0000003,36,36,36,36,36,36
nm0000004,50,50,50,50,50,50
nm0000005,6,6,6,6,6,6
...,...,...,...,...,...,...
nm9993699,1,1,1,0,0,0
nm9993700,1,1,1,0,0,0
nm9993703,1,1,1,0,0,0


In [50]:
actor_to_experience = {}
for row in total_actor_experience.iterrows():
    actor_to_experience[row[0]] = row[1]["tconst"]
del actor_to_experience[1] #remove the useless total count value there was

## Add column for age of actor during start of movie

In [9]:
# load in movie data and select relevent columns: tconst and startYear
movies_raw = pd.read_csv("data/title.basics.tsv", sep="\t")[["tconst", "startYear", "genres"]]

# filter out rows with missing data cruical for analysis
movies_raw = movies_raw[movies_raw["startYear"] != r"\N"]

/var/folders/dk/kt0tgs8j1cb1k_hn1_h2bch80000gn/T/ipykernel_4395/2564971414.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_raw = pd.read_csv("data/title.basics.tsv", sep="\t")[["tconst", "startYear", "genres"]]


In [10]:
# join movie data with actors
actor_movie_combi = principal_actors.merge(movies_raw, how="left", on="tconst")

# cast year columns to numeric in order to calculate
actor_movie_combi["startYear"] = pd.to_numeric(actor_movie_combi["startYear"]) # errors parameter for the case when date not given
actor_movie_combi["birthYear"] = pd.to_numeric(actor_movie_combi["birthYear"]) # errors parameter for the case when date not given
actor_movie_combi["deatYear"] = pd.to_numeric(actor_movie_combi["deathYear"], errors="coerce") # errors parameter for the case when actor is not dead, then NaN

# add age_at_movie_start column
actor_movie_combi["age_at_movie_start"] = actor_movie_combi["startYear"] - actor_movie_combi["birthYear"]
print("Number of movie actor combinations:", actor_movie_combi.size)

actor_movie_combi

Number of movie actor combinations: 230855020


,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...
20986815,tt9916856,4,nm10538647,1,NaN,NaN,NaN,2015.0,Short,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN
20986817,tt9916880,2,nm0254176,1,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN


## Add column for imdb ratings

In [11]:
# load in imdb ratings
ratings_raw = pd.read_csv('data/title.ratings.tsv', sep='\t')

In [12]:
# add ratings to overall data set
actor_movie_combi = actor_movie_combi.merge(ratings_raw, how="left", on="tconst")

actor_movie_combi

,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0,6.2,2554.0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0,6.2,2554.0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0,5.4,797.0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0,5.4,797.0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0,5.4,2069.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20986815,tt9916856,4,nm10538647,1,NaN,NaN,NaN,2015.0,Short,NaN,NaN,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN
20986817,tt9916880,2,nm0254176,1,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN


## Add column for number of movies an actor has been in before start of respective movie

In [25]:
actor_movie_combi["exp_so_far"] = 0
actor_movie_combi.sort_values(by="startYear", inplace=True, ascending=True)
from collections import defaultdict
exp_so_far = defaultdict(int)
for row in actor_movie_combi.iterrows():
    actor = row[1]["nconst"]
    actor_movie_combi.loc[row[0], "exp_so_far"] = exp_so_far[actor]
    exp_so_far[actor] += 1

# Drop rows with NA values and save to CSV
actor_movie_combi.dropna(inplace=True) 
actor_movie_combi.to_csv("data/actor_movie_combi.csv" , index=None)

In [14]:
actor_movie_combi

,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes,exp_so_far
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0,6.2,2554.0,0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0,6.2,2554.0,0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0,5.4,797.0,0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0,5.4,797.0,0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0,5.4,2069.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20986391,tt9916406,1,nm0682935,0,Ryszard Pietruski,1922.0,1996,1976.0,Crime,1996.0,54.0,5.8,5.0,34
20986392,tt9916406,2,nm0468565,0,August Kowalczyk,1921.0,2012,1976.0,Crime,2012.0,55.0,5.8,5.0,23
20986393,tt9916406,3,nm0387710,0,Zygmunt Hobot,1930.0,2004,1976.0,Crime,2004.0,46.0,5.8,5.0,3
20986395,tt9916406,9,nm0926802,0,Kazimierz Wichniarz,1915.0,1995,1976.0,Crime,1995.0,61.0,5.8,5.0,13


In [22]:
actor_movie_combi = pd.read_csv("data/actor_movie_combi.csv")
actor_movie_combi.sort_values(by="startYear", inplace=True, ascending=True)

## Add column for number of movies per genre an actor has been in before start of respective movie

In [26]:
# Will do major genres
genres_major = ['Drama', "Action", "Romance","Comedy"]
from collections import defaultdict
actor_movie_combi["drama_exp_so_far"] = 0
actor_movie_combi["action_exp_so_far"] = 0
actor_movie_combi["romance_exp_so_far"] = 0
actor_movie_combi["comedy_exp_so_far"] = 0
drama_exp_so_far = defaultdict(int)
action_exp_so_far = defaultdict(int)
romance_exp_so_far = defaultdict(int)
comedy_exp_so_far = defaultdict(int)

for row in actor_movie_combi.iterrows():
    actor = row[1]["nconst"]
    genres = row[1]["genres"].split(",")

    if "Drama" in genres:
        drama_exp_so_far[actor] += 1
    if "Action" in genres:
        action_exp_so_far[actor] += 1
    if "Romance" in genres:
        romance_exp_so_far[actor] += 1
    if "Comedy" in genres:
        comedy_exp_so_far[actor] += 1

    actor_movie_combi.loc[row[0], "drama_exp_so_far"] = drama_exp_so_far[actor]
    actor_movie_combi.loc[row[0], "action_exp_so_far"] = action_exp_so_far[actor]
    actor_movie_combi.loc[row[0], "romance_exp_so_far"] = romance_exp_so_far[actor]
    actor_movie_combi.loc[row[0], "comedy_exp_so_far"] = comedy_exp_so_far[actor]
    
actor_movie_combi.to_csv("data/actor_movie_combi.csv", index=None)

In [33]:
actor_movie_combi = pd.read_csv("data/actor_movie_combi.csv")

In [34]:
actor_movie_combi

,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes,exp_so_far,drama_exp_so_far,action_exp_so_far,romance_exp_so_far,comedy_exp_so_far
0,tt7816420,1,nm1155956,0,Eadweard Muybridge,1830.0,1904,1881.0,"Documentary,Short",1904.0,51.0,5.2,462.0,0,0,0,0,0
1,tt1758563,1,nm1796515,0,Adolphe Le Prince,1872.0,1901,1888.0,"Documentary,Short",1901.0,16.0,5.5,1333.0,0,0,0,0,0
2,tt0361921,1,nm1362928,0,Giuseppe Sacco Albanese,1872.0,1943,1890.0,"Documentary,Short",1943.0,18.0,5.0,1508.0,0,0,0,0,0
3,tt0416047,1,nm1362928,0,Giuseppe Sacco Albanese,1872.0,1943,1890.0,Short,1943.0,18.0,4.2,425.0,1,0,0,0,0
4,tt0416046,1,nm1362928,0,Giuseppe Sacco Albanese,1872.0,1943,1890.0,Short,1943.0,18.0,4.8,1106.0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698941,tt21093464,1,nm1048199,1,Jee-Young Kim,1938.0,2017,2022.0,Drama,2017.0,84.0,7.9,9.0,49,12,0,38,40
698942,tt21155694,7,nm4152681,0,Aleksey Gerilovich,1973.0,2022,2022.0,"Comedy,Drama",2022.0,49.0,7.2,51.0,1,1,0,0,2
698943,tt8442440,10,nm0878500,1,Mary Twala,1939.0,2020,2022.0,Drama,2020.0,83.0,6.2,23.0,6,6,0,0,0
698944,tt20599022,8,nm11512147,1,Kim Mi-Soo,1992.0,2022,2022.0,"Comedy,Fantasy,Romance",2022.0,30.0,8.6,20.0,11,2,0,11,12
